# JSON examples and exercise
****
+ get familiar with packages for dealing with JSON
+ study examples with JSON strings and files 
+ work on exercise to be completed and submitted 
****
+ reference: http://pandas.pydata.org/pandas-docs/stable/io.html#io-json-reader
+ data source: http://jsonstudio.com/resources/
****

In [ ]:
import pandas as pd

## imports for Python, Pandas

In [ ]:
import json
from pandas.io.json import json_normalize

## JSON example, with string

+ demonstrates creation of normalized dataframes (tables) from nested json string
+ source: http://pandas.pydata.org/pandas-docs/stable/io.html#normalization

In [ ]:
# define json string
data = [{'state': 'Florida', 
         'shortname': 'FL',
         'info': {'governor': 'Rick Scott'},
         'counties': [{'name': 'Dade', 'population': 12345},
                      {'name': 'Broward', 'population': 40000},
                      {'name': 'Palm Beach', 'population': 60000}]},
        {'state': 'Ohio',
         'shortname': 'OH',
         'info': {'governor': 'John Kasich'},
         'counties': [{'name': 'Summit', 'population': 1234},
                      {'name': 'Cuyahoga', 'population': 1337}]}]

In [ ]:
# use normalization to create tables from nested element
json_normalize(data, 'counties')

In [ ]:
# further populate tables created from nested element
json_normalize(data, 'counties', ['state', 'shortname', ['info', 'governor']])

****
## JSON example, with file

+ demonstrates reading in a json file as a string and as a table
+ uses small sample file containing data about projects funded by the World Bank 
+ data source: http://jsonstudio.com/resources/

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [ ]:
# load json as string
json.load((open('/kaggle/input/world_bank_projects_less.json')))

In [ ]:
# load as Pandas dataframe
sample_json_df = pd.read_json('/kaggle/input/world_bank_projects_less.json')
sample_json_df

****
## JSON exercise

Using data in file 'data/world_bank_projects.json' and the techniques demonstrated above,
1. Find the 10 countries with most projects
2. Find the top 10 major project themes (using column 'mjtheme_namecode')
3. In 2. above you will notice that some entries have only the code and the name is missing. Create a dataframe with the missing names filled in.

In [ ]:
full_json_df = pd.read_json('/kaggle/input/world_bank_projects.json')
full_json_df

In [ ]:
# 1.Find the 10 countries with most projects
count_df=full_json_df.groupby('countryname').count()[['sector']]
c=count_df.sort_values(by='sector', ascending=False)
r=c.reset_index()
r['countryname'][:10]


In [ ]:
#Find the top 10 major project themes (using column 'mjtheme_namecode')
themeJsonList = full_json_df['mjtheme_namecode'].tolist()
dfs=[json_normalize(themeJson)for themeJson in themeJsonList]
df_A=pd.concat(dfs, ignore_index=True)


In [ ]:
#top 10 
df_A.name.value_counts().sort_values(ascending = False).head(10)

In [ ]:
#3. In 2. above you will notice that some entries have only the code and the name is missing. Create a dataframe with the missing names filled in.

In [ ]:
#unique codes
df_A['code'].nunique()

In [ ]:
# unique combination of code and name 
unique_df = df_A.drop_duplicates()
unique_df = unique_df[unique_df.name != '']

unique_df = unique_df.reset_index().drop('index',axis=1)
unique_df

In [ ]:
filldf = pd.merge(df_A, unique_df, on='code', suffixes=('_x','')).drop('name_x' , axis=1)
filldf.head()

In [ ]:
filldf[filldf.name == ''].count() # no missing values